In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys,os
sys.path.append('..')
from backtester import matlab, backtester
from backtester.analysis import *
from backtester.strategy import StrategyBase, OptParam, OptParamArray
from backtester.swarms.ranking import SwarmRanker
from backtester.swarms.rebalancing import SwarmRebalance
from backtester.swarms.filters import SwarmFilter
from backtester.costs import CostsManagerEXOFixed
from backtester.exoinfo import EXOInfo
from backtester.swarms.rankingclasses import *
from backtester.swarms.swarm import Swarm

import statsmodels.tsa.api as smt
from sklearn import neighbors

from pandas.tseries.offsets import BDay

import pandas as pd
import numpy as np
import scipy

In [ ]:
from scripts.settings import *
try:
    from scripts.settings_local import *
except:
    pass

from exobuilder.data.exostorage import EXOStorage

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)


exo_filter = '*'     # All 
#exo_filter = 'ES_'  # ES only
exo_filter = 'CL'  # ES Collars (incl Bearish, Bullish, BW and vanilla)

[print(exo) for exo in storage.exo_list(exo_filter)];

In [ ]:
class Strategy_MLRegModel_KNN_pxfeatures(StrategyBase):
    name = 'Strategy_MLRegModel_KNN_pxfeatures'


    def __init__(self, strategy_context):
        # Initialize parent class
        super().__init__(strategy_context)
   
    def calc_entryexit_rules(self, prediction_days, rolling_median_window, rules_index):
        i = 100
        px_ser = self.data.exo
        empty_ser = pd.Series(False, index=px_ser.dropna().index)

        signals_df = pd.DataFrame()

        signals_df['entry_rule'] = empty_ser
        signals_df['exit_rule'] = empty_ser

        while True:
            dataset_df = pd.DataFrame(index=px_ser.dropna().iloc[:i].index)

            px_ser_slice = px_ser[dataset_df.index].rolling(rolling_median_window).median()

            dataset_df['target'] = px_ser_slice.shift(-prediction_days)

            #dataset_df['features_px'] = px_ser_slice
            dataset_df['features_shift1'] = px_ser_slice.shift(1)
            #dataset_df['features_shift2'] = px_ser_slice.shift(2)
            #dataset_df['features_shift3'] = px_ser_slice.shift(3)
            #dataset_df['features_shift5'] = px_ser_slice.shift(5)
            #dataset_df['features_shift10'] = px_ser_slice.shift(10)

            dataset_df['features_ma5'] = px_ser_slice.rolling(5).mean()
            #dataset_df['features_ma10'] = px_ser_slice.rolling(10).mean()
            #dataset_df['features_ma20'] = px_ser_slice.rolling(20).mean()
            #dataset_df['features_ma30'] = px_ser_slice.rolling(30).mean()

            dataset_df['features_relstr5'] = px_ser_slice - px_ser_slice.rolling(5).mean()
            #dataset_df['features_relstr10'] = px_ser_slice - px_ser_slice.rolling(10).mean()
            #dataset_df['features_relstr20'] = px_ser_slice - px_ser_slice.rolling(20).mean()
            #dataset_df['features_relstra30'] = px_ser_slice - px_ser_slice.rolling(30).mean()

            predict_features = dataset_df.filter(like='features').dropna()

            dataset_df.dropna(inplace=True)

            target = dataset_df.target

            features = dataset_df.filter(like='features')

            #model = linear_model.LinearRegression()
            #model = linear_model.ElasticNet()
            #model = linear_model.RANSACRegressor()

            #knn = neighbors.KNeighborsRegressor(5)
            #lr = linear_model.LinearRegression()
            #model = ensemble.BaggingRegressor(base_estimator=None, n_estimators=50)

            #model = linear_model.RANSACRegressor()
            #model = tree.DecisionTreeRegressor(max_depth=4)
            #model = svm.NuSVR()
            #model = ensemble.AdaBoostRegressor(n_estimators=10)
            #model = ensemble.GradientBoostingRegressor()
            #model = ensemble.RandomForestRegressor(n_estimators=50, max_depth=5, n_jobs=-1)
            model = neighbors.KNeighborsRegressor(5, algorithm='auto')
            #model = neighbors.RadiusNeighborsRegressor(5, n_jobs=1, algorithm='auto')

            model.fit(features.iloc[:i], target.iloc[:i])

            prediction_series = pd.Series(model.predict(features.iloc[:i]), 
                                          index=features.iloc[:i].index, name='Prediction') 

            if rules_index == 0:
                if prediction_series[-1] > prediction_series[-2]:
                    signals_df['entry_rule'].ix[px_ser_slice.index[-1]] = True


                    signals_df['exit_rule'].ix[px_ser_slice.index[-1] + BDay(1)] = True


            if rules_index == 1:
                if prediction_series[-1] < prediction_series[-2]:
                    signals_df['entry_rule'].ix[px_ser_slice.index[-1]] = True


                    signals_df['exit_rule'].ix[px_ser_slice.index[-1]+ BDay(1)] = True

            if i >= px_ser.dropna().size:
                break

            i += 1
        
        return signals_df.entry_rule, signals_df.exit_rule
    
    def calculate(self, params=None, save_info=False):
    #
    #
    #  Params is a tripple like (50, 10, 15), where:
    #   50 - slow MA period
    #   10 - fast MA period
    #   15 - median period
    #
    #  On every iteration of swarming algorithm, parameter set will be different.
    #  For more information look inside: /notebooks/tmp/Swarming engine research.ipynb
    #

        if params is None:
            # Return default parameters
            direction, prediction_days, rolling_median_window, rules_index = self.default_opts()
        else:
            # Unpacking optimization params
            #  in order in self.opts definition
            direction, prediction_days, rolling_median_window, rules_index = params

        # Defining EXO price
        px = self.data.exo
                
        entry_rule, exit_rule = self.calc_entryexit_rules(prediction_days, rolling_median_window, rules_index)

        # Swarm_member_name must be *unique* for every swarm member
        # We use params values for uniqueness
        swarm_member_name = self.get_member_name(params)

        #
        # Calculation info
        #
        calc_info = None
        if save_info:
            calc_info = {'trailing_stop': trailing_stop}

        return swarm_member_name, entry_rule, exit_rule, calc_info

## Script settings

In [ ]:
STRATEGY_CONTEXT = {
    'strategy': { 
        'class': Strategy_MLRegModel_KNN_pxfeatures,
        'exo_name': 'CL_PutSpread',        # <---- Select and paste EXO name from cell above
        'exo_storage': storage,          
        'opt_params': [
                        #OptParam(name, default_value, min_value, max_value, step)
                        OptParamArray('Direction', [1]),
                        OptParam('n Prediction Days', 1, 4, 10, 2),
                        OptParamArray('Price smoothing rolling median window', [1,5,7]),
                        OptParamArray('Rules index', np.arange(2)),
                        
            ],
    },
    'swarm': {
        'members_count': 2,
        'ranking_class': RankerBestWithCorrel(window_size=-1, correl_threshold=0.5),
        'rebalance_time_function': SwarmRebalance.every_friday,

    },
    'costs':{
        'manager': CostsManagerEXOFixed,
        'context': {
            'costs_options': 3.0,
            'costs_futures': 3.0,
        }
    }
}

# Backtest class based strategy

In [ ]:
smgr = Swarm(STRATEGY_CONTEXT)
smgr.run_swarm()
smgr.pick()

# Saving results to swarms directory
smgr.save('./swarms/')

### WARNING! Loading swarm from file (don't run next cell if you want new swarm instance)

In [ ]:
smgr.strategy.data.exo.plot()

In [ ]:
figsize(10,10)
smgr.picked_equity.plot(label='Picked swarm equity');
smgr.raw_equity.plot(label='Average swarm equity');
legend(loc=2);

In [ ]:
smgr.raw_swarm.plot(legend=False)

(smgr.strategy.data.exo*1).plot(linewidth=2, color='red')

In [ ]:
smgr.picked_swarm.plot()

smgr.picked_swarm.sum(1).plot(label='smgr.picked_swarm-sum')
smgr.picked_equity.plot(label='Picked swarm equity')

# Swarm exposure

In [ ]:
smgr.picked_exposure.sum(axis=1).rolling(10).mean().plot()

# Swarm statistics

#### Non-picked swarm stats

In [ ]:
smgr.picked_stats

## Exo information

In [ ]:
smgr.strategy.exoinfo.exo_info

In [ ]:
smgr.strategy.exoinfo.data.exo.plot()

### Global filter information (obsolete)


## Costs information (per 1-exo unit)

In [ ]:
figsize(10,5)
smgr.strategy.costs.plot()

## Margin graphs

### EXO Margin (per 1 EXO unit)

In [ ]:
smgr.strategy.exoinfo.margin().plot()

# Saving results

In [ ]:
smgr.save('./swarms/')

In [ ]:
smgr.raw_swarm#.plot()